# Laboratorio 2 - Reglas de Asociación y Patrones Secuenciales

## Entregable 2 - Patrones Secuenciales

Práctica realizada por: Belén Ortega Pérez, José Ignacio Navas Sanz, Paula Hipola Gómez

Importamos las librerias con las que trabajaremos.

In [1]:
import numpy as np                        #Esta libreria nos permite trabajar con numericos
import pandas as pd                       #Pandas nos permite trabajar con datasets y csv
import matplotlib.pyplot as plt           #Las dos librerias anteriores nos permiten pintar las funciones
from sklearn import preprocessing         #Nos sirve para normalizar los datos

from gsppy.gsp import GSP                     #Librería para poder implementar Generalized Sequential Patterns
from mlxtend.frequent_patterns import apriori #Libería relacionada con A priori

### 1. El conjunto de datos spotify_top10.csv contiene el ranking diario de las 10 canciones más reproducidas en Spotify cada día por país. Analizando esta información se pueden extraer conclusiones sobre cómo evoluciona la popularidad de las canciones. Este estudio se puede llevar a cabo aplicando el algoritmo Generalized Sequential Patterns utilizando la implementación de éste disponible en la librería gsppy. 
#### Prueba al menos dos configuraciones de soporte diferentes. Para una de ellas, interpreta algunos de los patrones secuenciales que te resulten curiosos. 

Añadimos el dataset con el que vamos a trabajar. En este caso vamos a utilizar una colección de datos que contiene el top 10 canciones escuchadas cada día por país. 

In [2]:
data = pd.read_csv('spotify_top10.csv')
data.head()                               #Función para ver las primeras instancias del dataset

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,ec
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,ec


Lo primero que debemos llevar a cabo es una limpieza del dataset. Para ello, primero averiguaremos con cuántos missing values y valores duplicados cuenta. 

**Datos duplicados**

In [3]:
duplicates_data = data.duplicated().sum()   #Cuenta el número de datos duplicados
print('Hay',duplicates_data,'datos duplicados')
data.drop_duplicates(inplace = True)        #Eliminamos los datos duplicados

Hay 0 datos duplicados


**Missing values**

In [4]:
missing_data1 = data.isnull().sum().sum()  #Cuenta el total de missing values en el dataset
missing_data2 = data.isnull().sum()        #Cuenta el total de missing values por atributo
print('Hay',missing_data1,' valores nulos')
print('\nDistribuido en columnas:')
print(missing_data2)

Hay 354  valores nulos

Distribuido en columnas:
Position        0
Track Name    177
Artist        177
Streams         0
URL             0
Date            0
Region          0
dtype: int64


Contamos con registros en los que el _Track Name_ y el _Artist_ son nulos. A pesar de tener estos valores vacíos, siempre tenemos una URL que nos lleva a una canción de Spotify. 

Hemos decidido copiar la URL de los registros con campos vacíos porque si aparecen en este conjunto de datos es porque en algún momento estubieron en la plataforma. Puede ser que hayan decidido dejar de compartir su contenido a Spotify y es por esto que no aparecen estos datos. 

In [5]:
#En los campos donde el Track Name es nulo, copiamos la URL.
data.loc[(data["Track Name"].isnull())&(data["URL"].notnull()),"Track Name"] = data["URL"]
data = data.replace(np.nan, '', regex=True)
#Volvemos a mostrar los datos nulos por atributo para demostrar que no quedan valores nulos en el nombre de la canción
missing_data2 = data.isnull().sum()
print(missing_data2)

Position      0
Track Name    0
Artist        0
Streams       0
URL           0
Date          0
Region        0
dtype: int64


In [11]:
changes = data.loc[data['Track Name'] == data['URL']]
changes.head()                                      #Mostramos ejemplos de como hemos suistituido los missing values

,Position,Track Name,Artist,Streams,URL,Date,Region
3094,5,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,16767,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-09,ec
3104,5,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,19114,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-10,ec
3115,6,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,21390,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-11,ec
3125,6,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,19644,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-12,ec
3136,7,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,16155,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-13,ec


En nuestra muestra de datos es muy importante la posición de 

In [7]:
transactions = data.groupby(['Date','Region'],as_index=False)['Track Name'].agg(lambda x: list(x))
transactions

,Date,Region,Track Name
0,2017-01-01,ar,"[Chantaje, Vente Pa' Ca, Reggaetón Lento (Bail..."
1,2017-01-01,at,"[Alone, Rockabye (feat. Sean Paul & Anne-Marie..."
2,2017-01-01,au,"[Starboy, Rockabye (feat. Sean Paul & Anne-Mar..."
3,2017-01-01,be,"[Hey Baby, Closer, Starboy, Rockabye (feat. Se..."
4,2017-01-01,bo,"[Reggaetón Lento (Bailemos), Otra Vez (feat. J..."
...,...,...,...
19670,2018-01-09,sv,"[Déjala Que Vuelva (feat. Manuel Turizo), Sens..."
19671,2018-01-09,tr,"[Yanıyoruz, Heyecanı Yok, Gömün Beni Çukura, r..."
19672,2018-01-09,tw,"[Wolves, 連名帶姓, So Far Away, 身後, 偷故事的人, Havana,..."
19673,2018-01-09,us,"[rockstar, Finesse (Remix) [feat. Cardi B], No..."


In [8]:
transactions_track = transactions['Track Name']
transactions_track

0        [Chantaje, Vente Pa' Ca, Reggaetón Lento (Bail...
1        [Alone, Rockabye (feat. Sean Paul & Anne-Marie...
2        [Starboy, Rockabye (feat. Sean Paul & Anne-Mar...
3        [Hey Baby, Closer, Starboy, Rockabye (feat. Se...
4        [Reggaetón Lento (Bailemos), Otra Vez (feat. J...
                               ...                        
19670    [Déjala Que Vuelva (feat. Manuel Turizo), Sens...
19671    [Yanıyoruz, Heyecanı Yok, Gömün Beni Çukura, r...
19672    [Wolves, 連名帶姓, So Far Away, 身後, 偷故事的人, Havana,...
19673    [rockstar, Finesse (Remix) [feat. Cardi B], No...
19674    [Échame La Culpa, Corazón, Criminal, Déjala Qu...
Name: Track Name, Length: 19675, dtype: object

In [9]:
evaluation = GSP(transactions_track).search(0.3)
evaluation

[{('Shape of You',): 10301}]

In [10]:
evaluation2 = GSP(transactions_track).search(0.1)
evaluation2

[{('I Don’t Wanna Live Forever (Fifty Shades Darker) - From "Fifty Shades Darker (Original Motion Picture Soundtrack)"',): 2960,
  ('Me Rehúso',): 4683,
  ('Shape of You',): 10301,
  ('Castle on the Hill',): 2154,
  ('Paris',): 2589,
  ('Despacito (Featuring Daddy Yankee)',): 3440,
  ('El Amante',): 2184,
  ("It Ain't Me (with Selena Gomez)",): 3205,
  ('Something Just Like This',): 3342,
  ('Symphony (feat. Zara Larsson)',): 2007,
  ('Ahora Dice',): 2156,
  ('Escápate Conmigo',): 2657,
  ('Despacito - Remix',): 5211,
  ('Felices los 4',): 2436,
  ('Unforgettable',): 2162,
  ("I'm the One",): 2905,
  ('Mama',): 2079,
  ('2U (feat. Justin Bieber)',): 2345,
  ('Mi Gente',): 3407,
  ('New Rules',): 3749,
  ('Havana',): 3641,
  ('Dusk Till Dawn - Radio Edit',): 2199,
  ('rockstar',): 3184}]